<a href="https://colab.research.google.com/github/Nikkuchan/BleepBot/blob/main/UnnathiBased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from keras.models import Model
from sklearn.metrics import classification_report
from keras import layers, models, optimizers
import sklearn.metrics as metrics
from sklearn.feature_extraction import text 


In [33]:
speech_data = pd.read_csv("profanitylabel.csv")

# split the dataset into training and testing datasets 
x_train, x_test, y_train, y_test = train_test_split(speech_data['Transcript'], speech_data['Category'], test_size=0.3, random_state=1)


# create a count vectorizer object 
my_additional_stop_words = ["akin","ako","mo","ka","tayo","ikaw","alin","amin",
                 "ay","bababa","dapat","ginawa","ginawang","bawat","ano",
                 "dahil","bilang","tama","na","nga","using","si","pati","ang",
                 "hala","pre","nag","edi","punta","pinupuntahan","lang","sa",
                 "mali","-","q","mong","date","busog","kayo","kayong","buka",
                 "ibubuka","nais","try","yung","katropa","gaya","ng","pic","bang",
                 "wala","pakita","computer","ambag","dito","nabawasan","bawas",
                 "footballer", "napaka","sayo","balota","bida","pietro","jett",
                 "mag-text","para","may","hindi","ni","ba","dalawa","salba","tutok",
                 "shadow", "install","ballaran","ito","mga","nalang","noh","baka",
                 "samin","mag","nasa","pa","masa","ata","po","ita","ga","kasi","eh",
                 "ol","nang","kang","naman","ko","tae","ata","atang","sana","kung","di",
                 "ah","much","pag"]

stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)
cv = CountVectorizer(stop_words='english')
cv.fit(speech_data['Transcript'])

# transform the training and testing data using count vectorizer object
x_train_count =  cv.transform(x_train)
x_test_count =  cv.transform(x_test)

tfidf_tr = TfidfTransformer()
x_train_count_tfidf = tfidf_tr.fit_transform(x_train_count)
x_test_count_tfidf = tfidf_tr.transform(x_test_count)

# ngram level tf-idf 
tv_ngram = TfidfVectorizer(ngram_range=(1,2))
tv_ngram.fit(speech_data['Transcript'])
x_train_tfidf_ngram =  tv_ngram.transform(x_train)
x_test_tfidf_ngram =  tv_ngram.transform(x_test)

In [34]:
speech_data['Category'].value_counts()

bobo         100
tanga        100
gago         100
fuck         100
putangina     53
tangina       47
Name: Category, dtype: int64

In [35]:
def classifierModel(classifier, Xtrain, Ytrain, Xtest):
    classifier.fit(Xtrain, Ytrain)
    predictions = classifier.predict(Xtest)
    print("\nAccuracy : ", accuracy_score(predictions, y_test))
    
    target_names = ['bobo', 'tanga','gago','fuck','putangina','tangina']
    print(classification_report(y_test, predictions, target_names=target_names))

In [36]:
print("\nNaive Bayes with Count Vectors: ")
classifierModel(MultinomialNB(), x_train_count, y_train, x_test_count)

print("\nNaive Bayes with Count Vectors + TF-IDF: ")
classifierModel(MultinomialNB(), x_train_count_tfidf, y_train, x_test_count_tfidf)

print("\nNaive Bayes with N-Gram Vectors: ")
classifierModel(MultinomialNB(), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)


Naive Bayes with Count Vectors: 

Accuracy :  0.9066666666666666
              precision    recall  f1-score   support

        bobo       0.83      0.89      0.86        28
       tanga       0.97      0.97      0.97        31
        gago       0.94      1.00      0.97        30
        fuck       1.00      0.65      0.79        20
   putangina       0.87      0.96      0.91        27
     tangina       0.86      0.86      0.86        14

    accuracy                           0.91       150
   macro avg       0.91      0.89      0.89       150
weighted avg       0.91      0.91      0.90       150


Naive Bayes with Count Vectors + TF-IDF: 

Accuracy :  0.7533333333333333
              precision    recall  f1-score   support

        bobo       0.68      0.89      0.77        28
       tanga       0.91      0.94      0.92        31
        gago       0.88      0.93      0.90        30
        fuck       1.00      0.10      0.18        20
   putangina       0.59      0.96      0.73  

In [37]:
print("\nLinearSVM with Count Vectors: ")
classifierModel(SVC(kernel='linear'), x_train_count, y_train, x_test_count)

print("\nLinearSVM with Count Vectors + TF-IDF: ")
classifierModel(SVC(kernel='linear'), x_train_count_tfidf, y_train, x_test_count_tfidf)

print("\nLinearSVM with N-Gram Vectors: ")
classifierModel(SVC(kernel='linear'), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)


LinearSVM with Count Vectors: 

Accuracy :  0.96
              precision    recall  f1-score   support

        bobo       1.00      0.89      0.94        28
       tanga       0.86      1.00      0.93        31
        gago       1.00      1.00      1.00        30
        fuck       1.00      0.85      0.92        20
   putangina       1.00      1.00      1.00        27
     tangina       0.93      1.00      0.97        14

    accuracy                           0.96       150
   macro avg       0.97      0.96      0.96       150
weighted avg       0.97      0.96      0.96       150


LinearSVM with Count Vectors + TF-IDF: 

Accuracy :  0.9733333333333334
              precision    recall  f1-score   support

        bobo       1.00      0.96      0.98        28
       tanga       0.97      1.00      0.98        31
        gago       1.00      1.00      1.00        30
        fuck       0.95      0.90      0.92        20
   putangina       0.93      1.00      0.96        27
     tang

In [38]:
print("\nRandom Forrest Classifier with Count Vectors: ")
classifierModel(RandomForestClassifier(random_state=40,n_estimators=1024), x_train_count, y_train, x_test_count)

print("\nRandom Forrest Classifier with Count Vectors + TF-IDF: ")
classifierModel(RandomForestClassifier(random_state=40,n_estimators=1024), x_train_count_tfidf, y_train, x_test_count_tfidf)

print("\nRandom Forrest Classifier with N-Gram Vectors: ")
classifierModel(RandomForestClassifier(random_state=40,n_estimators=1024), x_train_tfidf_ngram, y_train, x_test_tfidf_ngram)


Random Forrest Classifier with Count Vectors: 

Accuracy :  0.9533333333333334
              precision    recall  f1-score   support

        bobo       0.96      0.89      0.93        28
       tanga       0.94      1.00      0.97        31
        gago       1.00      1.00      1.00        30
        fuck       0.94      0.85      0.89        20
   putangina       0.96      1.00      0.98        27
     tangina       0.87      0.93      0.90        14

    accuracy                           0.95       150
   macro avg       0.95      0.95      0.94       150
weighted avg       0.95      0.95      0.95       150


Random Forrest Classifier with Count Vectors + TF-IDF: 

Accuracy :  0.9533333333333334
              precision    recall  f1-score   support

        bobo       1.00      0.89      0.94        28
       tanga       0.91      1.00      0.95        31
        gago       1.00      1.00      1.00        30
        fuck       0.94      0.85      0.89        20
   putangina     